In [ ]:
!pip install -U bert-extractive-summarizer
!pip install -U python-docx
!pip install -U transformers

     |████████████████████████████████| 1.8MB 5.5MB/s 
     |████████████████████████████████| 890kB 21.6MB/s 
     |████████████████████████████████| 2.9MB 18.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=244c590ba80320a83606dcb7124d5bd5b28107376f25cf2ac1b2b1d4a934035a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 5.5MB 5.3MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=f99cf6d618df36fd4a50e20b18ae37eaed31b9a6cc2ab4cf73857ae5d74f5e71
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx
Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.2)
Requirement already up-to-date: pymystem3 in /usr/local/lib/python3.6/dist-packages 

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-02-07 07:51:30--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.241, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-02-07 07:51:31--  http://cache-mskm910.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)... 5.45.220.20, 2a02:6b8:0:2002::1b
Connecting to cache-mskm910.cdn.yandex.net (cache-mskm910.cdn.yandex.net)|5.45.220.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [ ]:
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE

--2021-02-07 07:15:48--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30447 (30K) [text/plain]
Saving to: ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’

people.csv?token=AI 100%[===================>]  29.73K  --.-KB/s    in 0.003s  

2021-02-07 07:15:48 (10.2 MB/s) - ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’ saved [30447/30447]

--2021-02-07 07:15:48--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [ ]:
import requests
import pandas as pd
from docx import Document
from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string
from pymystem3 import Mystem
import re
import math


nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.append('рис')
russian_stopwords.append('т.к.')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
url = 'http://lms.hse.ru/ap_service.php?getwork=1&guid=D6A3C344-D35B-4820-8514-EF33F306E5E3'
r = requests.get(url, allow_redirects=True)

open('test.docx', 'wb').write(r.content)
f = open('test.docx', 'rb')
document = Document(f)

In [ ]:
fullText = ""
for para in document.paragraphs:
    fullText+=(para.text)+" "
print(fullText)

Пермь, 2018 год 
 Аннотация В этой работе описан процесс реализации виртуального собеседника с помощью лексико-синтаксических шаблонов с использованием языка LSPL.  Для разработанного виртуального собеседника реализована система администрирования. Работа содержит 3 главы, 13 иллюстраций и 4 приложения.   
 Оглавление                            Введение Современное продуктивное обучение предполагает интенсивную работу как со стороны студентов, так и со стороны преподавателей, в том числе и выполнение большого количества проектов для усвоения полученных знаний на практике. Зачастую такие проекты требуют больших затрат времени как со стороны преподавателя, так и со стороны студентов. В этом случае такие проекты должны быть автоматизированы, хотя бы частично. Об автоматизации одного из таких учебных проектов и пойдет речь в этой работе.  В ходе работы будет улучшен разработанный ранее виртуальный собеседник для деловой игры «Развертывание» по технологии MSF. В ходе деловой игры студенты де

In [ ]:
custom_config = AutoConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)

In [ ]:
def lowerCase(Corpus):
    corpus = "".join([i.lower().replace('\n',' ') for i in Corpus])
    return corpus

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(Corpus):
    """custom function to remove the punctuation"""
    return "".join([i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in Corpus])

def remove_stopwords(Corpus):
    """custom function to remove the stopwords"""
    return " ".join([token for token in Corpus.split() if token not in russian_stopwords])

def remove_freqwords(Corpus):
    cnt = Counter()
    for text in Corpus:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(30)])
    return " ".join([word for word in str(Corpus).split() if word not in FREQWORDS])

def remove_urls(Corpus):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    return_text=[]
    for text in Corpus:
        text=url_pattern.sub(r'', text)
        text=html_pattern.sub(r'',text)
        return_text.append(text)
    return "".join(i for i in return_text)
    
def remove_numbers(Corpus):
    text = ["".join(i for i in text if not i.isdigit()) for text in Corpus]
    return "".join(i for i in text)

def Stem(Corpus):
    stemmer = SnowballStemmer("russian") 
    return "".join(m.lemmatize(Corpus))

def Lemm(Corpus):
    m = Mystem()
    return "".join(m.lemmatize(Corpus))

def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 

def clearTextAfterGarbage(Corpus):
    corpus = Corpus
    corpus = lowerCase(Corpus)
    corpus = remove_urls(corpus)
    corpus = remove_numbers(corpus)
    corpus = remove_stopwords(corpus)
    #corpus = remove_punctuation(corpus)
    corpus = Lemm(corpus)
    #corpus = Stem(corpus)
    corpus = remove_freqwords(corpus)
    return corpus

In [ ]:
text = clearTextAfterGarbage(fullText)
text

'пермь, год аннотация работа описывать процесс реализация виртуальный собеседник помощь лексико-синтаксический шаблон использование язык lspl. разрабатывать виртуальный собеседник реализовать система администрирование. работа содержать глава, иллюстрация приложение. оглавление введение современный продуктивный обучение предполагать интенсивный работа сторона студент, сторона преподаватель, число выполнение большой количество проект усвоение получать знание практика. зачастую такой проект требовать большой затрата время сторона преподаватель, сторона студент. случай такой проект должный автоматизировать, хотя частично. автоматизация один такой учебный проект пойти речь работа. ход работа улучшать разрабатывать ранее виртуальный собеседник деловой игра «развертывание» технология msf. ход деловой игра студент делиться группа, группа должный опрашивать сотрудник отдел фирма филиал четыре город, разбрасывать весь мир. каждый сотрудник обладать свой компетенция знать весь информация филиал, 

In [ ]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(fullText, min_length=20, max_length=100)
summary = "".join(result)
print(len(summary), summary)

3115 Для разработанного виртуального собеседника реализована система администрирования. Актуальность работы: работа является актуальной, т.к. Объект работы: процесс проведения. Предмет работы: автоматизация консультационной работы преподавателя в ходе проекта. Первая глава содержит описание предметной области и анализ автоматизируемого процесса. Planning (Планирование). На этапе разработки пишется код приложения, подготавливается документация. Stabilizing (Стабилизация). В ходе игры студенты должны установить вымышленную систему «ОбалдеИТ» на компьютеры пользователей. Формирование расписания развертывания. Трудозатраты преподавателя. Выявление наличия в указанном филиале фирмы сотрудника с указанной должностью. Определение ответа сотрудника на заданный вопрос. Возможность просмотра заданных вопросов. Полный список требований к системе представлен в приложении А. Глава 2. Искусственные нейронные сети (ИНС). Попытка использования первой версии собеседника выявила следующие проблемы:  Кол

In [ ]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(text, min_length=20, max_length=100)
summary = "".join(result)
print(len(summary), summary)

2525 разрабатывать виртуальный собеседник реализовать система администрирование. преподаватель это трудозатраты, студент - затрата время игра целое это длительность. работа разделять глава. developing (разработка). этап проводиться тестирование приложение, устраняться обнаруживать ошибка. ход игра студент должный устанавливать вымышленный система «обалдеит» компьютер пользователь. анализ игра разбирать процесс проведение игра рамка обучение студент. общение преподаватель одновременно доступно сразу несколько группа студент. трудозатраты преподаватель. преподаватель вынуждать отвечать весь студент многочисленный вопрос, требовать значительный усилие. готовый входной файл содержать запись. практика вариант пробовать реализовать. исходить следствие теорема арнольд–колмогоров-хехт–нильсен[], большой вероятность переобучение инс. лексико-синтаксический шаблон (лсш). русский язык это падеж, время, наклонение, склонение, число т.д. попытка использование первый версия собеседник выявлять следу

In [ ]:
vkr = pd.read_csv('/content/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE')
vkr.head()

,Unnamed: 0,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file
0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN
1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...
2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN
3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN
4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN


In [ ]:
vkrText = vkr
vkrText['fullText'] = ''
vkrText['shortText'] = ''

In [166]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        try:
            open('test.docx', 'wb').write(r.content)
            f = open('test.docx', 'rb')
            document = Document(f)
            f.close()
            successful.append(url)
        except:
            unsuccessful.append(url)
            vkrText.loc[row, 'fullText' ] = ''
            vkrText.loc[row, 'shortText'] = ''
            continue
        
        fullText = ""
        for para in document.paragraphs:
            fullText+=(para.text)+" "
        vkrText.loc[row, 'fullText' ] = fullText
        '''
        text = clearTextAfterGarbage(fullText)
        result = modelSum(fullText, min_length=20, max_length=100)
        summary = "".join(result)

        vkrText.loc[row, 'shortText'] = summary
        '''
    else:
        vkrText.loc[row, 'link_file'] = ''
        vkrText.loc[row, 'fullText' ] = ''
        vkrText.loc[row, 'shortText'] = ''

100%|█████████▉| 1702/1708 [20:43<00:10,  1.74s/it]





100%|█████████▉| 1703/1708 [20:47<00:13,  2.61s/it]





100%|█████████▉| 1705/1708 [20:49<00:06,  2.09s/it]





100%|█████████▉| 1706/1708 [20:52<00:04,  2.36s/it]





100%|██████████| 1708/1708 [20:54<00:00,  1.36it/s]


In [167]:
len(successful), len(unsuccessful)

(437, 72)

In [168]:
vkrText.to_csv('vkrText.csv')